# Sentiment Analysis

Think for this exploration I will primarily be drawing from _Opinion Digger: An Unsupervised Opinion Miner from Unstructured Product Reviews_.

### Load the Data

In [1]:
import pandas as pd

def load_data():
    article_table = pd.read_csv("../data/raw/kaggle1/articles1.csv")
    return article_table

In [2]:
articles = load_data()

### Load the Aspects

In [9]:
import json

aspect_data = {}
pos_sentences = []
with open("../data/cache/kaggle_aspects/aspects.json") as in_file:
    aspect_data = json.load(in_file)

with open("../data/cache/kaggle_aspects/pos.json") as in_file:
    pos_sentences = json.load(in_file)

In [10]:
aspect_data

{'republicans': {'count': 80,
  'sentences': [0,
   2,
   3,
   4,
   6,
   10,
   12,
   18,
   19,
   21,
   23,
   1654,
   1657,
   1696,
   1698,
   1703,
   1712,
   1713,
   1715,
   1718,
   1720,
   1721,
   1728,
   1731,
   1733,
   1746,
   2374,
   2376,
   2383,
   2388,
   2413,
   2426,
   2429,
   2487,
   2494,
   2495,
   2495,
   2507,
   2518,
   2519,
   2520,
   2527,
   2534,
   2547,
   2559,
   2617,
   3271,
   3272,
   3272,
   3274,
   3281,
   3283,
   3291,
   3301,
   3302,
   3303,
   3314,
   3346,
   3353,
   3383,
   3383,
   3387,
   3388,
   3389,
   3389,
   3399,
   3408,
   4512,
   4948,
   5030,
   5031,
   5394,
   5405,
   5416,
   5428,
   5440,
   5478,
   5482,
   5483,
   5522],
  'pos': [['republicans', 'NNS']],
  'flr': 992.7738916792686},
 'fear': {'count': 14,
  'sentences': [0,
   55,
   86,
   1836,
   2704,
   2712,
   2915,
   3591,
   3646,
   4019,
   4972,
   5113,
   5535,
   5536],
  'pos': [['fear', 'NN']],
  'flr': 68.5857

In [11]:
pos_sentences

[[['washington', 'JJ'],
  ['congressional', 'JJ'],
  ['republicans', 'NNS'],
  ['have', 'VBP'],
  ['a', 'DT'],
  ['new', 'JJ'],
  ['fear', 'NN'],
  ['when', 'WRB'],
  ['it', 'PRP'],
  ['comes', 'VBZ'],
  ['to', 'TO'],
  ['their', 'PRP$'],
  ['health', 'NN'],
  ['care', 'NN'],
  ['lawsuit', 'NN'],
  ['against', 'IN'],
  ['the', 'DT'],
  ['obama', 'JJ'],
  ['administration', 'NN'],
  ['they', 'PRP'],
  ['might', 'MD'],
  ['win', 'VB']],
 [['the', 'DT'],
  ['incoming', 'JJ'],
  ['trump', 'NN'],
  ['administration', 'NN'],
  ['could', 'MD'],
  ['choose', 'VB'],
  ['to', 'TO'],
  ['no', 'DT'],
  ['longer', 'JJR'],
  ['defend', 'VB'],
  ['the', 'DT'],
  ['executive', 'NN'],
  ['branch', 'NN'],
  ['against', 'IN'],
  ['the', 'DT'],
  ['suit', 'NN'],
  ['which', 'WDT'],
  ['challenges', 'VBZ'],
  ['the', 'DT'],
  ['administration', 'NN'],
  ['authority', 'NN'],
  ['to', 'TO'],
  ['spend', 'VB'],
  ['billions', 'NNS'],
  ['of', 'IN'],
  ['dollars', 'NNS'],
  ['on', 'IN'],
  ['health', 'NN'],
  

### Aspect Pruning

In [16]:
pruned_data = {}

for aspect in aspect_data.keys():
    if aspect_data[aspect]["flr"] > 10.0:
        pruned_data[aspect] = aspect_data[aspect]


print(len(aspect_data))
print(len(pruned_data))

19495
4086


### Find closest adjectives to each aspect

In [24]:
for aspect in pruned_data.keys():
    pos_aspect = pruned_data[aspect]["pos"]
    for sentence_index in pruned_data[aspect]["sentences"]:
        sentence = pos_sentences[sentence_index]
        
        # find the index of the aspect
        aspect_index = sentence.index(pos_aspect[0])
        
        # iterate on each side
        for i in range(1, 6):
            left = aspect_index - i
            if left >= 0:
                pass
    break

2
27
16
5
7
11
13
11
5
22
5
2
5
3
1
17
7
9
5
3
4
3
7
14
5
1
10
4
1
1
6
8
5
10
24
5
5
14
8
0
4
16
6
1
9
6
13
10
10
9
3
1
2
6


ValueError: ['republicans', 'NNS'] is not in list